In [ ]:
import cv2
import copy
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
from torchvision.transforms import functional as F
import os
from src.tools.utils import write_json, clear_file, is_video_detect, find_next, find_reference
from src.tools.VideoClip import VideoClip
from src.models.PoseDetect import PoseDetect
from src.models.CourtDetect import CourtDetect
from src.models.NetDetect import NetDetect
import argparse
from src.tools.BallDetect import ball_detect
import warnings
# clear the polyfit Rankwarning
warnings.simplefilter('ignore', np.RankWarning)

result_path="C:/Users/86153/Desktop/res"
name_list=[
    "Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals",
    "Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_BWF_World_Championships_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_England_Open_2022_Semi_finals",
    "An_Se_Young_Chen_Yu_Fei_PERODUA_Malaysia_Masters_2022_Finals",
    "AN_Se_Young_Gregoria_Mariska_TUNJUNG_Malaysia_Masters_2022_SemiFinals",
    "AN_Se_Young_Pornpawee_CHOCHUWONG_Korea_Open_Badminton_Championships_2022_Finals",
    "AN_Seyoung_PUSARLA_V",
    "Anders_Antonsen_Viktor_Axelsen_HSBC_BWF_WORLD_TOUR_FINALS_2020_Finals",
    "Anthony_Sinisuka_Ginting_Rasmus_Gemke_YONEX_Thailand_Open_2021_QuarterFinals",
]
for video_name in name_list:
    print("-" * 10 + "Starting Ball Detection" + "-" * 10)       
    for res_root, res_dirs, res_files in os.walk(
            f"{result_path}/videos/{video_name}"):
        for res_file in res_files:
            _, ext = os.path.splitext(res_file)
            if ext.lower() in ['.mp4']:
                res_video_path = os.path.join(res_root, res_file)
                print(res_video_path)
                ball_detect(res_video_path, f"{result_path}/ball")
    print("-" * 10 + "End Badminton Detection" + "-" * 10)


In [ ]:
import torch

print(torch.__version__)

In [ ]:
import cv2
import copy
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
from torchvision.transforms import functional as F
import os
from src.tools.utils import write_json, read_json,clear_file, is_video_detect, find_next, find_reference
from src.tools.VideoClip import VideoClip
from src.models.PoseDetect import PoseDetect
from src.models.CourtDetect import CourtDetect
from src.models.NetDetect import NetDetect
import argparse
from src.tools.BallDetect import ball_detect
import warnings
# clear the polyfit Rankwarning
warnings.simplefilter('ignore', np.RankWarning)

result_path="E:\\res"
name_list=[
    "Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals",
    "Akane_YAMAGUCHI_AN_Se_Young_DAIHATSU_YONEX_Japan_Open_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_BWF_World_Championships_2022_Finals",
    "Akane_YAMAGUCHI_CHEN_Yu_Fei_England_Open_2022_Semi_finals",
    "An_Se_Young_Chen_Yu_Fei_PERODUA_Malaysia_Masters_2022_Finals",
    "AN_Se_Young_Gregoria_Mariska_TUNJUNG_Malaysia_Masters_2022_SemiFinals",
    "AN_Se_Young_Pornpawee_CHOCHUWONG_Korea_Open_Badminton_Championships_2022_Finals",
    "AN_Seyoung_PUSARLA_V",
    "Anders_Antonsen_Viktor_Axelsen_HSBC_BWF_WORLD_TOUR_FINALS_2020_Finals",
    "Anthony_Sinisuka_Ginting_Rasmus_Gemke_YONEX_Thailand_Open_2021_QuarterFinals",
]
print(result_path)
for video_name in name_list:
    print("-" * 10 + "Starting Ball Detection" + "-" * 10)       
    json_path=os.path.join(f"{result_path}/players/player_kp",f"{video_name}.json")
    print(json_path)
    # break

    info_dict=read_json(json_path)
    player_dict={}
    # 遍历 JSON 对象中的每个键值对
    for key, value in info_dict.items():
        if value['top'] is None or value['bottom'] is None:
            continue
        player_dict[key]={
            'top':value['bottom'],
            'bottom':value['top']
        }

        write_json(player_dict, video_name,f"E:/ans/players/player_kp")

    print("-" * 10 + "End Badminton Detection" + "-" * 10)
